In [52]:
import pandas as p
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [53]:
data = p.read_csv("cleaned_train.csv")
data.head()

,Unnamed: 0,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,1,0,3,1,22.0,1,0,7.2500,2
1,1,2,1,1,0,38.0,1,0,71.2833,0
2,2,3,1,3,0,26.0,0,0,7.9250,2
3,3,4,1,1,0,35.0,1,0,53.1000,2
4,4,5,0,3,1,35.0,0,0,8.0500,2


In [54]:
un_label = ["Unnamed: 0","PassengerId"]
data = data.drop(labels= un_label, axis = 1)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [55]:
lt = [[0,0],[0,1]]
lt = np.array(lt)

In [56]:
sur = []
for index,s in data.iterrows():
    if int(s["Survived"]) == 1:
        #print(True)
        sur.append(lt[1])
    if int(s["Survived"]) == 0:
        #print(False)
        sur.append(lt[0])

In [57]:
data["Survived"] =sur

In [58]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,"[0, 0]",3,1,22.0,1,0,7.2500,2
1,"[0, 1]",1,0,38.0,1,0,71.2833,0
2,"[0, 1]",3,0,26.0,0,0,7.9250,2
3,"[0, 1]",1,0,35.0,1,0,53.1000,2
4,"[0, 0]",3,1,35.0,0,0,8.0500,2


In [59]:
details = data.drop(axis = 1, labels="Survived")
label = np.array(sur)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(details, label, test_size=0.25, random_state=42)

In [61]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [62]:
X_train

array([[  1.        ,   1.        ,  29.69911765, ...,   0.        ,
         30.5       ,   2.        ],
       [  3.        ,   1.        ,  25.        , ...,   0.        ,
          7.05      ,   2.        ],
       [  2.        ,   0.        ,  24.        , ...,   2.        ,
         14.5       ,   2.        ],
       ...,
       [  3.        ,   1.        ,  41.        , ...,   0.        ,
         14.1083    ,   2.        ],
       [  1.        ,   0.        ,  14.        , ...,   2.        ,
        120.        ,   2.        ],
       [  1.        ,   1.        ,  21.        , ...,   1.        ,
         77.2875    ,   2.        ]])

In [63]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
scaler.fit(X_test)
X_test = scaler.transform(X_test)
scaler.fit(y_train)
y_train = scaler.transform(y_train)
scaler.fit(y_test)
y_test = scaler.transform(y_test)

C:\Users\Charans\AppData\Local\conda\conda\envs\py36\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [64]:
n_nodes_hl1 = 500
n_nodes_hl2 = 250
n_nodes_hl3 = 100
n_nodes_hl4 = 50
n_classes = 2
batch_size = 25

In [65]:
x = tf.placeholder('float')
y = tf.placeholder('float')

In [66]:
def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([len(X_train[0]),n_nodes_hl1])), 
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])), 
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])), 
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    hidden_4_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_nodes_hl4])), 
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl4]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl4,n_classes])), 
                      'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']),hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']),hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']),hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    l4 = tf.add(tf.matmul(l3, hidden_4_layer['weights']),hidden_4_layer['biases'])
    l4 = tf.nn.relu(l4)
    
    output = tf.matmul(l4, output_layer['weights']) + output_layer['biases']
    
    return(output, hidden_1_layer, hidden_2_layer, hidden_3_layer, hidden_4_layer,output_layer)

In [69]:
def train_neural_network(x):
    prediction,w1,w2,w3,w4,ol = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    hm_epochs = 50
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            i=0
            while i < len(X_train):
                start = i
                end = i+batch_size
                batch_x = X_train[start:end]
                batch_y = y_train[start:end]

                _,c = sess.run([optimizer,cost], feed_dict={x:batch_x,y:batch_y})
                epoch_loss += c
                i+=batch_size
            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
        
        saver.save(sess,"./titan.ckpt")        
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        
        accuracy = tf.reduce_mean(tf.cast(correct,'float')) 
        
        print('accuracy', accuracy.eval({x:X_test, y:y_test}))
        #print(sess.run([w1['weights'],w2['weights'],w3['weights'],w4['weights']]))

In [70]:
train_neural_network(x)

Epoch 0 completed out of 50 loss: 38569.66520375013
Epoch 1 completed out of 50 loss: 2120.682582616806
Epoch 2 completed out of 50 loss: 1253.6368227005005
Epoch 3 completed out of 50 loss: 1033.4075546562672
Epoch 4 completed out of 50 loss: 2210.168677330017
Epoch 5 completed out of 50 loss: 1569.589033484459
Epoch 6 completed out of 50 loss: 2304.212498664856
Epoch 7 completed out of 50 loss: 2569.382749557495
Epoch 8 completed out of 50 loss: 5494.439994812012
Epoch 9 completed out of 50 loss: 6207.309960842133
Epoch 10 completed out of 50 loss: 20594.923553466797
Epoch 11 completed out of 50 loss: 45349.90010070801
Epoch 12 completed out of 50 loss: 22821.839814662933
Epoch 13 completed out of 50 loss: 47782.719985961914
Epoch 14 completed out of 50 loss: 85028.74618530273
Epoch 15 completed out of 50 loss: 121322.56079101562
Epoch 16 completed out of 50 loss: 75040.15984344482
Epoch 17 completed out of 50 loss: 100709.76065826416
Epoch 18 completed out of 50 loss: 420010.9912109

In [71]:
model = neural_network_model(x)
restorer=tf.train.Saver()
with tf.Session() as sess1:
    restorer.restore(sess1,"./titan.ckpt")
    test_input = tf.cast(X_train[0:5],tf.float32)
    sess1.run(model, feed_dict = {x:test_input})

INFO:tensorflow:Restoring parameters from ./titan.ckpt


NotFoundError: Key Variable_120 not found in checkpoint
	 [[Node: save_8/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_8/Const_0_0, save_8/RestoreV2/tensor_names, save_8/RestoreV2/shape_and_slices)]]

Caused by op 'save_8/RestoreV2', defined at:
  File "C:\Users\Charans\AppData\Local\conda\conda\envs\py36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Charans\AppData\Local\conda\conda\envs\py36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Charans\AppData\Local\conda\conda\envs\py36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Charans\AppData\Local\conda\conda\envs\py36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Charans\